In [1]:
import pandas as pd
import random

# Charger les données à partir des fichiers CSV
train_df = pd.read_csv('test.csv')

# Créer une liste de tous les utilisateurs uniques
unique_users = train_df['user'].unique()

# Initialiser la liste pour stocker les paires de tweets
tweet_pairs = []

# Déterminer le nombre maximum de paires à générer pour chaque utilisateur
max_pairs_per_user = len(train_df) // len(unique_users)

# Boucler à travers chaque utilisateur
for user in unique_users:
    # Extraire tous les tweets de l'utilisateur actuel
    user_tweets = train_df[train_df['user'] == user]['text'].tolist()
    
    # Générer des paires de tweets à partir du même utilisateur
    same_user_pairs = random.sample(list(zip(user_tweets, user_tweets)), max_pairs_per_user)
    
    # Extraire les tweets des autres utilisateurs
    other_users_tweets = train_df[train_df['user'] != user]['text'].tolist()
    
    # Sélectionner aléatoirement un nombre équivalent de tweets des autres utilisateurs
    random_other_tweets = random.sample(other_users_tweets, max_pairs_per_user)
    
    # Créer des paires de tweets avec des utilisateurs différents
    different_user_pairs = list(zip(user_tweets, random_other_tweets))
    
    # Ajouter les paires de tweets à la liste avec une étiquette de similarité 1 pour les mêmes utilisateurs
    tweet_pairs.extend([[pair[0], pair[1], 1] for pair in same_user_pairs])
    
    # Ajouter les paires de tweets à la liste avec une étiquette de similarité 0 pour les différents utilisateurs
    tweet_pairs.extend([[pair[0], pair[1], 0] for pair in different_user_pairs])

# Mélanger les paires de tweets pour assurer un ordre aléatoire
random.shuffle(tweet_pairs)

# Convertir en DataFrame
tweet_pairs_df = pd.DataFrame(tweet_pairs, columns=['tweet1', 'tweet2', 'similarity_label'])

# Afficher un aperçu du DataFrame
print(tweet_pairs_df.head())

# Enregistrer le DataFrame dans un fichier CSV
tweet_pairs_df.to_csv('test_tweet_pairs.csv', index=False)

                                              tweet1  \
0  @JustBe74 V important to make all of humanity ...   
1  All North American dates on #THEPRISMATICWORLD...   
2  Here's an in-depth look at how our economy has...   
3  The @Bucks run out to the court as they look t...   
4  CAN'T WAIT to watch @kathybethterry's online c...   

                                              tweet2  similarity_label  
0  Crooked Hillary no longer has credibility - to...                 0  
1  All North American dates on #THEPRISMATICWORLD...                 1  
2  Pixar’s new movie will hit you right in the fe...                 0  
3  Leonardo Ulloa seals Brighton return\n\nMore o...                 0  
4  CAN'T WAIT to watch @kathybethterry's online c...                 1  


In [2]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# Télécharger les ressources nécessaires pour NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Charger le DataFrame
df = pd.read_csv('test_tweet_pairs.csv')

# Initialiser le lemmatiseur
lemmatizer = WordNetLemmatizer()

# Fonction de nettoyage du texte avec lemmatisation
def clean_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer la ponctuation, les symboles et les autres caractères spéciaux sauf les hashtags et les mentions
    text = ''.join([char if char in ['#', '@'] or char not in string.punctuation else ' ' for char in text])
    # Tokenization
    tokens = word_tokenize(text)
    # Supprimer les mots vides (stop words)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatisation
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Rejoindre les tokens pour former une chaîne de texte
    cleaned_text = ' '.join(lemmatized_tokens)
    return cleaned_text

# Appliquer la fonction de nettoyage du texte à chaque tweet dans le DataFrame
df['tweet1'] = df['tweet1'].apply(clean_text)
df['tweet2'] = df['tweet2'].apply(clean_text)

# Afficher un aperçu du DataFrame après le prétraitement
print(df.head())

# Enregistrer le DataFrame prétraité dans un nouveau fichier CSV
df.to_csv('preprocessed_test_tweet_pairs.csv', index=False)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khadidja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khadidja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\khadidja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                              tweet1  \
0  @ justbe74 v important make humanity proud cas...   
1  north american date # theprismaticworldtour sa...   
2  depth look economy made real progress last six...   
3  @ buck run court look get 11th win 14 game str...   
4  wait watch @ kathybethterry online chat @ www ...   

                                              tweet2  similarity_label  
0  crooked hillary longer credibility much failur...                 0  
1  north american date # theprismaticworldtour sa...                 1  
2  pixar ’ new movie hit right feel http goo gl v...                 0  
3  leonardo ulloa seal brighton return loan deal ...                 0  
4  wait watch @ kathybethterry online chat @ www ...                 1  
